In [1030]:
import pandas as pd
from kss import split_sentences
from pykospacing import Spacing
from hanspell import spell_checker
from konlpy.tag import Okt, Kkma, Hannanum, Komoran, Mecab
import re

from typing import List
from konlpy.tag import Okt
from textrankr import TextRank

### Lemmatizer Process

In [964]:
total_df = pd.read_csv('dataset/강아지 가족.csv', encoding = 'utf-8-sig')

In [965]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Q_title      912 non-null    object
 1   Q_content    757 non-null    object
 2   A_content_1  912 non-null    object
 3   A_content_2  612 non-null    object
 4   A_content_3  367 non-null    object
 5   A_content_4  183 non-null    object
 6   A_content_5  83 non-null     object
dtypes: object(7)
memory usage: 51.5+ KB


In [966]:
total_df=total_df[['Q_content']].dropna(axis=0)

In [967]:
total_df.reset_index(drop=True, inplace=True)

In [968]:
total_df

,Q_content
0,제가 너무 스트레스 받아서 제가 이상한건지 여쭤봅니다처음 강아지 입양 했을때 물론 ...
1,저는 8년동안 페키니즈를 키우고 있습니다.저희 페키는 배변훈련을 어렸을때부터 해서 ...
2,"안녕하세요 강아지 1인 가족 일 시, 잘 키울 수 있는 방법? (강아지 훈련 이나 ..."
3,어머니가 약 3달 전에 아기 강아지를 80만원 가량을 지불하고 데려오셨습니다. 저희...
4,안녕하세요 제가 진짜로 예전 부터 강아지를 엄청 좋아했는데요...그런데못 키우는 이...
...,...
752,안녕하세요 6개월된 남아 말티즈 키우는 대학생입니다. 이 아이를 데려온지 4개월이 ...
753,안녕하세요 저는 미니어쳐 푸들을 키우고 있는 고등학생입니다. 다름이 아니라 저희...
754,혹시 강아지 입양처 구하고 계시는 분 있으면 댓 달아주세요 저희는 4인가족이고 강아...
755,안녕하세요평범한 소녀입니다ㅋ강아지를 키우려는데 첨이거든요;;그래서 뭐를 사야할지 어...


### 데이터 품사 분류

In [969]:
for i in range(len(total_df['Q_content'])):
    try:
        norm = re.compile('[가-힣|0-9|.|?]+').findall(total_df.loc[i,'Q_content'])
        st = ' '.join(norm)
        st2 = spell_checker.check(st)
        total_df.loc[i,'Q_content']=st2[2]
    except:
        print(i)

In [970]:
komoran=Komoran()
total_df['Q_title_pos']=total_df['Q_content'].map(lambda x: komoran.pos(x))

In [971]:
print(total_df['Q_title_pos'][1])

[('저', 'NP'), ('는', 'JX'), ('8', 'SN'), ('년', 'NNB'), ('동안', 'NNG'), ('페키니즈', 'NNP'), ('를', 'JKO'), ('키우', 'VV'), ('고', 'EC'), ('있', 'VX'), ('습니다', 'EF'), ('.', 'SF'), ('저희', 'NP'), ('페리', 'NNP'), ('는', 'JX'), ('배변', 'NNP'), ('훈련', 'NNG'), ('을', 'JKO'), ('어리', 'VA'), ('었', 'EP'), ('을', 'ETM'), ('때', 'NNG'), ('부터', 'JX'), ('하', 'VV'), ('아서', 'EC'), ('애견', 'NNG'), ('카페나', 'NNP'), ('다른', 'MM'), ('곳', 'NNG'), ('을', 'JKO'), ('가도', 'NNP'), ('배변', 'NNP'), ('패드', 'NNP'), ('에', 'JKB'), ('잘', 'MAG'), ('볼일', 'NNG'), ('을', 'JKO'), ('보', 'VV'), ('았', 'EP'), ('는데', 'EC'), ('이사', 'NNG'), ('오', 'VV'), ('고', 'EC'), ('몇', 'MM'), ('달', 'NNG'), ('이', 'JKS'), ('지나', 'VV'), ('니까', 'EC'), ('애가', 'NNP'), ('계속', 'MAG'), ('자기', 'NP'), ('화장실', 'NNG'), ('에서', 'JKB'), ('변', 'NNG'), ('을', 'JKO'), ('보', 'VV'), ('지', 'EC'), ('않', 'VX'), ('습니다', 'EF'), ('.', 'SF'), ('저희', 'NP'), ('가족', 'NNG'), ('화장실', 'NNP'), ('바로', 'MAG'), ('옆', 'NNG'), ('에', 'JKB'), ('강아지', 'NNP'), ('화장실', 'NNG'), ('이', 'JKS'), ('있', 'VV'), ('는데', '

### 형용사, 동사의 용언을 원형 복원 하기

https://github.com/lovit/korean_lemmatizer 참고
<br>!pip install soylemma

default dictionary는 세종 말뭉치 dataset 기준

In [995]:
from soylemma import Lemmatizer, hangle
lemmatizer = Lemmatizer(dictionary_name='default')

In [996]:
okt=Okt()
total_df['Q_title_pos']=total_df['Q_content'].map(lambda x: okt.pos(x))

In [997]:
cnt_lem={}
cnt_lem_data={}
exp_data=[]
for i in range(len(total_df['Q_title_pos'])):
    for st, ty in total_df.loc[i,'Q_title_pos']:
        if (ty=='Verb' or ty=='Adjective') and len(st)!=1 : # 1글자는 제외하기 (위에서 판단한 결과)
            list_st=lemmatizer.lemmatize(st)
            if len(list_st)==1:
                st_lem=list_st[0][0]
                if len(st_lem) not in cnt_lem.keys():
                    cnt_lem[len(st_lem)]=0
                    cnt_lem_data[len(st_lem)]=[st_lem]
                else:
                    cnt_lem[len(st_lem)]+=1
                    if st_lem not in cnt_lem_data[len(st_lem)]:  # 중복제거
                        cnt_lem_data[len(st_lem)].append(st_lem)
            else:
                if st not in exp_data:
                    exp_data.append(st)
cnt_lem=sorted(cnt_lem.items())
cnt_lem_data=sorted(cnt_lem_data.items())

In [998]:
cnt_lem

[(2, 1985), (3, 1740), (4, 528), (5, 109), (6, 10)]

In [999]:
# 2글자 형용사 동사 용언 정리 (중복 제거)
print(cnt_lem_data[0])

(2, ['틀다', '같다', '않다', '보다', '짖다', '없다', '자다', '뛰다', '많다', '싶다', '알다', '오다', '밀다', '막다', '맞다', '넘다', '들다', '떨다', '먹다', '받다', '닫다', '짓다', '작다', '가다', '울다', '뜯다', '찾다', '맡다', '씻다', '쏟다', '닦다', '내다', '낳다', '놀다', '주다', '열다', '갖다', '나다', '대다', '죽다', '살다', '읽다', '쉬다', '물다', '앉다', '쉽다', '두다', '믿다', '잡다', '개다', '새다', '뺏다', '입다', '깔다', '높다', '묻다', '삼다', '놓다', '시다', '꾸다', '치다', '빼다', '굶다', '밟다', '붙다', '쫓다', '말다', '사다', '얼다', '뜨다', '넣다', '풀다', '듣다', '꽂다', '늘다', '줍다', '긁다', '떼다', '웃다', '남다', '참다', '좁다', '돌다', '깎다', '추다', '밝다', '줄다', '벌다', '밉다', '굴다', '묶다', '깨다', '팔다', '눕다'])


In [1000]:
print(cnt_lem_data[1])

(3, ['키우다', '빠지다', '잘나다', '시키다', '여쭈다', '나가다', '보이다', '나오다', '만나다', '흔들다', '만지다', '아니다', '해주다', '싸우다', '지내다', '스럽다', '밀치다', '가보다', '가리다', '기르다', '다치다', '지키다', '고치다', '올리다', '피우다', '다니다', '멈추다', '가지다', '아프다', '놔두다', '날리다', '걸리다', '들이다', '놀라다', '먹이다', '적이다', '만들다', '때리다', '던지다', '아끼다', '삼키다', '여기다', '힘들다', '지나다', '해보다', '박히다', '병들다', '무섭다', '해오다', '그리다', '드리다', '덮이다', '계시다', '돌보다', '따르다', '이쁘다', '버리다', '안기다', '슬프다', '원하다', '비싸다', '깨물다', '파이다', '미치다', '사납다', '덤비다', '비우다', '내밀다', '반기다', '재밌다', '옮기다', '맡기다', '혼내다', '손대다', '나쁘다', '다르다', '뺐기다', '모이다', '잡히다', '똑같다', '급하다', '찢기다', '푸르다', '봐주다', '귀엽다', '사주다', '지겹다', '그렇다', '사보다', '보내다', '쳐놓다', '반갑다', '들리다', '부리다', '잠들다', '달리다', '빠르다', '바뀌다', '팔리다', '재우다', '모으다', '끝나다', '배우다', '씌우다', '아물다', '섞이다', '씻기다', '예쁘다', '굶기다', '서럽다', '망치다', '놓치다', '남기다', '살피다', '챙기다', '지리다', '느끼다', '바라다', '잠자다', '지르다', '외롭다', '더럽다', '할퀴다', '괴롭다', '안주다', '빨갛다', '담그다', '되찾다', '가두다', '조르다', '오지다', '두렵다', '뿐이다', '죽이다', '맞추다', '치이다', '하얗다', '싸가다', '입히다', '심하다', '내주다', '써주다', '

In [1001]:
print(cnt_lem_data[2])

(4, ['들어가다', '열어놓다', '달려들다', '시켜주다', '놀아주다', '돌아오다', '싫어하다', '돌아가다', '다가오다', '데려오다', '들어오다', '알려주다', '알아보다', '따라가다', '기다리다', '길러지다', '낳아주다', '다가가다', '태어나다', '주무시다', '간지럽다', '찢어지다', '순해지다', '찡그리다', '멀어지다', '친해지다', '망설이다', '뛰어놀다', '떨어지다', '심해지다', '어떡하다', '내버리다', '낑낑대다', '물어보다', '데려가다', '괴롭히다', '이뻐하다', '해드리다', '따라오다', '붉어지다', '도와주다', '몰라주다', '가려내다', '그러가다', '도망가다', '다녀오다', '머무르다', '반겨주다', '시끄럽다', '속상하다', '치워주다', '씻어주다', '깔아주다', '챙겨주다', '긁어주다', '움직이다', '달라지다', '쫓아오다', '없어지다', '우울하다', '하잖않다', '알아채다', '책임지다', '오잖않다', '찾아주다', '보내주다', '일어서다', '읽어주다', '안쓰럽다', '엎드리다', '살아나다', '맞춰주다', '징그럽다', '헤어지다', '돌봐주다', '맡겨놓다', '놀다가다', '달려오다', '내려놓다', '들려주다', '한눈팔다', '있잖않다', '골라주다', '심심하다', '가르치다', '조그맣다', '뽑아주다', '기뻐하다', '찾아보다', '올려놓다', '둘러보다', '고쳐지다', '만져지다', '아무렇다', '바라보다', '길들이다', '쳐다보다', '마주치다', '나타나다', '보여주다', '돌아서다', '남겨주다', '알아듣다', '나아지다', '날아가다', '옮겨지다', '꺼내주다', '열어주다', '던져주다', '자유롭다', '넣어놓다', '벌여지다', '깨어나다', '따라주다', '찾아오다', '올라오다', '가져오다', '잘해주다', '일어나다', '옮겨놓다', '찍어놓다', '많아지다', '살아가다', '먹여주다', '풀어

In [1002]:
# 5개 이상짜리
for i in range(3,100):
    try:
        print(cnt_lem_data[i])
    except: 
        print("없음")
        break

(5, ['부탁드리다', '돌아다니다', '모르잖않다', '그리워하다', '만들어주다', '잡아당기다', '낯설어하다', '미끄러지다', '갑작스럽다', '무서워하다', '쫓아다니다', '알아차리다', '잃어버리다', '따라다니다', '사나워지다', '내보내지다', '게을러지다', '귀여워하다', '뛰어나가다', '뛰어다니다', '두려워하다', '누워계시다', '괜찮아지다', '받아들이다', '잊어버리다', '말씀드리다', '끌어당기다'])
(6, ['쓰다듬어주다', '받아들여주다'])
없음


In [1003]:
print(exp_data)

['있습니다', '어렸을', '해서', '봤는데', '오고', '않습니다', '있는데도', '없습니다', '없어서', '나면', '가서', '하고', '괴롭습니다', '이러는', '고치는', '안녕하세요', '있는', '되는', '있나요', '해야', '하는지', '한다면', '도와주는', '이런', '했다고', '하는데', '되나요', '인해서', '있지는', '돼서', '걸린다는', '있어요', '되는지', '안되는지', '알려주시면', '감사하겠습니다', '했어요', '지을', '어떻게', '할까', '불안한', '건지도', '모르겠고', '있다면', '늦게', '나가셨는데', '그런', '건지', '그러는지', '있긴', '심해서', '고쳐지거든요', '있어도', '이러면서', '하는', '하고요', '돼가는데', '하면', '짖더라고요', '있을', '그러더라고요', '친하지', '주거든요', '심한', '모르겠어요', '물렸어요', '반기거든요', '누워있', '있으면', '알려주세요', '좋아하면서', '흔들다가도', '지나가는', '들면', '무는', '앙하고', '씹어요', '세게', '부족하고', '있다고', '일까', '아니면', '오냐오냐', '키운', '까요', '될지', '입니다', '있어야', '사는데', '그러죠', '불러도', '와서', '서운하고', '궁금해요', '있는데요', '새로운', '하더라고요', '좋을까요', '있던', '하러', '달려와서', '느끼', '느껴져서', '찍으려고요', '해주시면', '감사합니다', '가려고', '들었는데', '가능한가요', '부탁드려요', '집고', '서있고', '그러는데', '돼는데', '좋아하는', '했는데', '접는', '나서요', '왔었거든요', '메고', '잡아당기셨어요', '건지는', '모르겠는데', '됐거든요', '됐어요', '괜찮은데', '들어갔다', '나오거나', '나갔다', '잡거나', '키우거든요', '가려요', '해요', '지나가는데요', '있었습니다', '했다는

In [1004]:
# 필터링 되지 않았던 것(exp_data)는 VA+VV 작업을 거친다. 
komoran=Komoran()
def lemmatize(word):
    morphtags = komoran.pos(word)
    if morphtags[0][1] == 'VA' or morphtags[0][1] == 'VV':
        return morphtags[0][0] + '다'
    else:
        return 'None'

In [1005]:
exp2_data=[]
cnt_lem_data2=[]
for name in exp_data:
    temp=lemmatize(name)
    if temp=='None':
        exp2_data.append(name)
    else:
        if temp not in cnt_lem_data2:
            cnt_lem_data2.append(temp)

In [1006]:
print(cnt_lem_data2)

['있다', '어리다', '하다', '보다', '오다', '없다', '나다', '가다', '괴롭다', '이렇다', '고치다', '되다', '도와주다', '인하다', '걸리다', '알리다', '어떻다', '모르다', '늦다', '나가다', '걸다', '그렇다', '심하다', '짖다', '친하다', '물리다', '반기다', '눕다', '좋아하다', '흔들다', '지나가다', '들다', '물다', '씹다', '세다', '일다', '키우다', '까다', '이다', '살다', '부르다', '새롭다', '좋다', '달려오다', '느끼다', '찍다', '집다', '서다', '접다', '나서다', '메다', '잡아당기다', '건지다', '괜찮다', '들어가다', '나오다', '잡다', '가리다', '안타깝다', '반가워하다', '날뛰다', '남기다', '다가오다', '놀다', '싫어하다', '부리다', '자다', '쫓아오다', '생기다', '누르다', '들어오다', '먹다', '남다', '듣다', '빠지다', '적다', '시키다', '길다', '많다', '둥글다', '싸우다', '힘들다', '풀다', '춥다', '붙다', '떨어뜨리다', '가르치다', '날리다', '원하다', '보이다', '지나다', '혼내다', '어렵다', '걸치다', '알아보다', '안기다', '내려가다', '낑낑대다', '달려가다', '슬프다', '그립다', '속상하다', '마르다', '맛있다', '때리다', '데려오다', '떨어지다', '귀하다', '그리워하다', '맵다', '태어나다', '만지다', '자라다', '약하다', '해보다', '팔다', '맞다', '따르다', '앉다', '숨다', '잇다', '떼다', '핥다', '쌓다', '줄이다', '사다', '쓰다', '넓다', '벗다', '검다', '낳다', '쓰다듬다', '엄청나다', '바쁘다', '다가가다', '알다', '싫다', '맡기다', '상관없다', '만나다', '깨물다', '강하다', '바라다', '추리다', '오래되다', '좁다', '

In [1007]:
print(exp2_data)

['않습니다', '안녕하세요', '이런', '안되는지', '감사하겠습니다', '지을', '불안한', '건지도', '그런', '주거든요', '앙하고', '부족하고', '아니면', '서운하고', '궁금해요', '느끼', '해주시면', '감사합니다', '가능한가요', '부탁드려요', '왔었거든요', '있었습니다', '고잔', '였습니다', '비슷하다면', '살다가', '굉장히', '필요해서', '평범한', '보다보다', '스러울', '내는', '냈는데도', '미안하고', '곤란한', '자는', '어떤', '안되고', '않더라고요', '있었어요', '꿈이었네요', '찜찜한', '오셔서', '만만하게', '책임지', '주고', '멀쩡해요', '불안하고', '미안하네요', '불안해하는', '거든요', '미안하기도', '애틋하게', '궁금하네요', '있겠지만', '영리한', '집은', '그래요', '이상해요', '그럼', '그러더러', '싶다고', '겠죠', '주었거든요', '주었는데', '주면', '똑똑하다고', '똑똑한', '있으', '무지해서', '정확한', '안전한', '자란', '지에', '미안한', '주면서', '궁금해져서요', '궁금합니다', '자세한', '궁금한', '실패했습니다', '싶습니다', '있었으면', '싶다는', '강력하게', '어떠', '순해요', '더라고요', '있었는데', '주는', '줄까', '이상한', '싶고요', '필요한지', '지어', '행복하겠는데', '예민해지는', '궁금하고요', '행복하시고', '안되다', '있었지만', '진데', '아무런', '반기고요', '않으셔서요', '않아', '활발한', '인해', '아까까지만', '다심하게', '때려', '그러나요', '정확히는', '이상한가요', '건강해요', '가능하면', '겁니다', '였으면', '아니었으면', '어린아이도', '조용한', '간단히', '어린아이가', '조용하지', '적당할', '안되고요', '있었음', '않고요', '이뻐하고', '지가', '빌거리', '뭐라', 

In [1008]:
komoran.pos('사주신다네요')

[('사주', 'NNP'), ('신', 'NNG'), ('달', 'VV'), ('네요', 'EC')]

In [1009]:
komoran.pos('말인데요')

[('말', 'NNG'), ('이', 'VCP'), ('ㄴ데요', 'EC')]

### 추가로 제외된 용언 원형 분석 하기

방법론 (komoran을 이용한 형태소 분석을 통해 원형 분석)

 1. 'XR'(어근)+'XSA'(형용사파생접미사) 일 경우, 'XR'+'XSA'+'다' 를 붙인다.
 2. 'NNG'(보통명사)+'XSV'(동사파생접미사) 일 경우, 'NNG'+'XSV'+'다' 를 붙인다.
 3. 'NNG'(보통명사)+'XSA'(형용사파생접미사) 일 경우, 'NNG'+'XSA'+'다' 를 붙인다.
 4. 'VX'(보조용언)이 있는 경우 'VX'+'다' 를 붙인다.
 5. 'MAG'(일반부사)+'VV'(동사) 일 경우 'MAG'+'VV'+'다'를 붙인다.
 6. 'MAG'(일반부사)+'XSV'(동사파생접미사) 일 경우 'MAG'+'XSV'+'다'를 붙인다.
 7. 'NNG'(일반부사)+'VV'(동사) 일 경우 'NNG'+'VV'+'다'를 붙인다.

In [1010]:
exp3_data=[]
cnt_lem_data3=[]
for name in exp2_data:
    pumsa=komoran.pos(name)
    temp=''
    check=0
    for i in range(1,len(pumsa)):
        if len(pumsa)==1:  # 하나로 나뉘는 것은 의미 없음.
            break
            
        if pumsa[i-1][1]=='XR' and pumsa[i][1]=='XSA': # 1번째 방법론
            temp=str(pumsa[i-1][0])+str(pumsa[i][0])
            break
            
        if pumsa[i-1][1]=='NNG' and pumsa[i][1]=='NNG': # 2번째 방법론 
            temp=temp+str(pumsa[i-1][0])+str(pumsa[i][0])
            check+=1
        elif pumsa[i-1][1]=='NNG' and pumsa[i][1]=='XSV': # 2번째 방법론 이어서
            temp2=''
            for k in range(check):
                temp2+=pumsa[k][0]
            temp=temp2+str(pumsa[i-1][0])+str(pumsa[i][0])
            break
            
        if pumsa[i-1][1]=='NNG' and pumsa[i][1]=='XSA': # 3번째 방법론
            temp=name[:i-1]+str(pumsa[i-1][0])+str(pumsa[i][0])
            break
        
        if pumsa[i-1][1]=='VX':  # 4번째 방법론
            for k in range(i):
                temp+=pumsa[k][0]
            break
        
        if pumsa[i-1][1]=='MAG' and pumsa[i][1]=='VV':  # 5번째 방법론
            temp=name[:i-1]+str(pumsa[i-1][0])+str(pumsa[i][0])
            break
            
        if pumsa[i-1][1]=='MAG' and pumsa[i][1]=='XSV':  # 6번째 방법론
            temp=name[:i-1]+str(pumsa[i-1][0])+str(pumsa[i][0])
            break
            
        if pumsa[i-1][1]=='NNG' and pumsa[i][1]=='VV':  # 7번째 방법론
            temp=name[:i-1]+str(pumsa[i-1][0])+str(pumsa[i][0])
            break
            
    if len(temp)!=0:
        temp+='다'
        if temp not in cnt_lem_data3:
            cnt_lem_data3.append(temp)
    else:
        exp3_data.append(name)

In [1011]:
print(cnt_lem_data3)

['않다', '안되다', '감사하다', '지다', '불안하다', '주다', '앙하다', '부족하다', '서운하다', '궁금하다', '가능하다', '부탁드리다', '오다', '있다', '비슷하다', '필요하다', '평범하다', '내다', '미안하다', '곤란하다', '찜찜하다', '만만하다', '멀쩡하다', '애틋하다', '영리하다', '이상하다', '싶다', '하다', '똑똑하다', '무지하다', '정확하다', '안전하다', '자세하다', '실패하다', '강력하다', '순하다', '행복하다', '예민하다', '활발하다', '건강하다', '조용하다', '적당하다', '죄송하다', '단순하다', '버리다', '난감하다', '온순하다', '어떡하다', '놓다', '웬만하다', '흥분하다', '생생하다', '적응하다', '어떠하다', '정당하다', '너무하다', '귀엽다', '찌우자다', '유일하다', '심심하다', '불쌍하다', '민망하다', '얌전하다', '확실하다', '진지하다', '환장하다', '따뜻하다', '억울하다', '풍부하다', '완전하다', '심각하다', '착실하다', '우울하다', '고소하다', '야속하다', '남자아이다', '중요하다', '평화롭다', '어색하다', '완벽하다', '잘빠지다', '저렴하다', '울긋불긋하다', '깨끗하다', '소중하다', '통통하다', '불편하다', '답답하다', '깔끔하다', '부담스럽다', '성공하다', '소심하다', '그만하다', '사랑스럽다', '친절하다', '마땅하다', '특이하다', '성장하다', '잘하다', '애매하다', '만족하다', '신중하다', '함께하다', '말다', '뻔하다', '시큰둥하다', '전화하다', '특별하다', '익숙하다', '화나다', '진정하다', '지않다', '잔인하다', '성실하다', '쓸쓸하다', '한심하다', '따끔하다', '사소하다', '잠잠하다', '시원하다', '안데다', '황당하다', '위험하다', '간절하다', '다양하다', '그러하다', '특정하다', '막막하다'

In [1012]:
print(exp3_data)

['안녕하세요', '이런', '건지도', '그런', '아니면', '느끼', '해주시면', '고잔', '였습니다', '살다가', '굉장히', '보다보다', '스러울', '내는', '자는', '어떤', '꿈이었네요', '책임지', '거든요', '집은', '그래요', '그럼', '그러더러', '있으', '자란', '지에', '어떠', '더라고요', '아무런', '반기고요', '인해', '아까까지만', '다심하게', '때려', '그러나요', '정확히는', '겁니다', '였으면', '아니었으면', '어린아이도', '간단히', '어린아이가', '이뻐하고', '지가', '빌거리', '뭐라', '솔직히', '그런지', '계시고요', '뭐라고', '남자구여', '식구인데도', '찝찝하고', '였어요', '잖아요', '이니', '짜리', '신중히', '있다로', '물고요', '아닌데요', '여기는', '하도', '하셔요', '아니', '였고', '지나도다가', '삼주', '였는데', '해서라면', '여아인데', '하질', '자였던', '그러더니', '나가', '적이셨는데', '조용히', '안자면서', '어린아이나', '싶으데', '상당히', '하긴', '아닌데', '이겠지만', '느낌이네요', '단이네요', '하구', '양해도', '딱히', '분이시면', '그리고요', '완전히', '아파서요', '자야', '원해요', '좋아서', '독해서인가요', '고데기', '고데기에', '비벼대', '아파서', '입니다나이는', '아니더라고요', '이뻐해', '해달라는', '였을', '친해지는', '셈이죠', '해가', '그런데도', '특별히', '용로', '아니라면', '마세요', '마시고요', '아까', '충분히', '어느덧', '아니래요', '하진', '한대', '부르', '아닌가', '그런대', '느낌이었고요', '집이었습니다', '꿈이었을까요', '예뻐하고', '쏜살같이', '모아', '겨줄', '확실히', '할만', '만지', '자세히', '화나서', '말인데요', '사다', '

In [1013]:
for name in exp3_data:
    print(name,'\t',komoran.pos(name))

안녕하세요 	 [('안녕하세요', 'NNP')]
이런 	 [('이런', 'MM')]
건지도 	 [('건', 'NNB'), ('지도', 'NNP')]
그런 	 [('그런', 'MM')]
아니면 	 [('아니', 'VCN'), ('면', 'EC')]
느끼 	 [('느끼', 'XR')]
해주시면 	 [('해주시', 'NNP'), ('이', 'VCP'), ('면', 'EC')]
고잔 	 [('고자', 'NNP'), ('ㄴ', 'JX')]
였습니다 	 [('이', 'VCP'), ('었', 'EP'), ('습니다', 'EC')]
살다가 	 [('살다가', 'NNP')]
굉장히 	 [('굉장히', 'MAG')]
보다보다 	 [('보다', 'MAG'), ('보다', 'JKB')]
스러울 	 [('스럽', 'XSA'), ('ㄹ', 'ETM')]
내는 	 [('내', 'NP'), ('는', 'JX')]
자는 	 [('자', 'NNB'), ('는', 'JX')]
어떤 	 [('어떤', 'MM')]
꿈이었네요 	 [('꿈', 'NNG'), ('이', 'VCP'), ('었', 'EP'), ('네요', 'EC')]
책임지 	 [('책임', 'NNP'), ('지', 'NNB')]
거든요 	 [('거든요', 'EC')]
집은 	 [('집', 'NNG'), ('은', 'JX')]
그래요 	 [('그래요', 'IC')]
그럼 	 [('그럼', 'MAJ')]
그러더러 	 [('그', 'MM'), ('러', 'NNP'), ('더러', 'JKB')]
있으 	 [('있으', 'NA')]
자란 	 [('자란', 'NNP')]
지에 	 [('지', 'NNB'), ('에', 'JKB')]
어떠 	 [('어떠', 'XR')]
더라고요 	 [('더라고요', 'EC')]
아무런 	 [('아무런', 'MM')]
반기고요 	 [('반기', 'NNP'), ('고요', 'NNP')]
인해 	 [('인해', 'NNP')]
아까까지만 	 [('아까', 'NNG'), ('까지', 'JX'), ('만', 'JX')]
다심하

In [1031]:
from konlpy.tag import Okt, Komoran
from soylemma import Lemmatizer   # !pip install soylemma

# 동사, 형용사별로 입력 받을 시 용언의 원형으로 리턴
def Lemmatizer_NA(string):
    okt=Okt()
    st, ty=okt.pos(string)
    
    if (ty=='Verb' or ty=='Adjective') and len(st)!=1: # 1글자는 제외하기 (판단 결과)
        lemmatizer = Lemmatizer(dictionary_name='default')   
        list_st=lemmatizer.lemmatize(st)                       # 1번째 원형 복귀 방법
        
        if len(list_st)==1: #lemmatizer로 바로 인식 하면 리턴
            return str(list_st[0][0])
            
        else:
            komoran=Komoran()
            morphtags = komoran.pos(st) 
            if morphtags[0][1] == 'VA' or morphtags[0][1] == 'VV':  # 2번째 원형 복귀 방법
                return morphtags[0][0] + '다'  # 1번째 방법 틀릴 경우 VA+VV 작업을 거친다. 

            else:
                temp=''
                check=0
                for i in range(1,len(morphtags)):                   
                    if len(morphtags)==1:  # 하나로 나뉘는 것은 의미 없음.
                        break

                    if morphtags[i-1][1]=='XR' and morphtags[i][1]=='XSA': # 3번째 원형 복귀 방법
                        temp=str(morphtags[i-1][0])+str(morphtags[i][0])
                        break

                    if morphtags[i-1][1]=='NNG' and morphtags[i][1]=='NNG':   # 4번째 원형 복귀 방법 
                        temp=temp+str(morphtags[i-1][0])+str(morphtags[i][0])
                        check+=1
                    elif morphtags[i-1][1]=='NNG' and morphtags[i][1]=='XSV': # 4번째 원형 복귀 방법 (이어서)
                        temp2=''
                        for k in range(check):
                            temp2+=morphtags[k][0]
                        temp=temp2+str(morphtags[i-1][0])+str(morphtags[i][0])
                        break

                    if morphtags[i-1][1]=='NNG' and morphtags[i][1]=='XSA':    # 5번째 원형 복귀 방법
                        temp=st[:i-1]+str(morphtags[i-1][0])+str(morphtags[i][0])
                        break

                    if morphtags[i-1][1]=='VX':                            # 6번째 원형 복귀 방법
                        for k in range(i):
                            temp+=morphtags[k][0]
                        break

                    if morphtags[i-1][1]=='MAG' and morphtags[i][1]=='VV':     # 7번째 원형 복귀 방법
                        temp=string[:i-1]+str(morphtags[i-1][0])+str(morphtags[i][0])
                        break

                    if morphtags[i-1][1]=='MAG' and morphtags[i][1]=='XSV':    # 8번째 원형 복귀 방법
                        temp=string[:i-1]+str(morphtags[i-1][0])+str(morphtags[i][0])
                        break

                    if morphtags[i-1][1]=='NNG' and morphtags[i][1]=='VV':     # 9번째 원형 복귀 방법
                        temp=string[:i-1]+str(morphtags[i-1][0])+str(morphtags[i][0])
                        break

                if len(temp)!=0:
                    temp+='다'
                    return temp
                
                else:
                    return str(string)   # 이래도 안될 경우.. 그대로 return
    
    else:
        return str(string)

In [1032]:
Lemmatizer_NA('시킨')

ValueError: not enough values to unpack (expected 2, got 1)